In [1]:
from mesa.experimental import JupyterViz

from render import make_render_station_vehicle_count_chart, render_model_with_bg, render_model_orientations
from capacity_vanet import VECModel, get_grid

In [2]:
page = JupyterViz(
    VECModel,
    #model_params={"width": 100, "height": 80, "speed": 40},
    model_params={"width": 200, "height": 200, "speed": 40, "max_capacity": 25},
    measures=[
        # render_model,
        render_model_with_bg(get_grid()),
        # render_distance_chart, 
        make_render_station_vehicle_count_chart(tail=100),
        #render_model_orientations,
        "TotalSuccessfulHandoverCount",
        "TotalFailedHandoverCount",
        "AvgQoS",
        "MinQoS",
        "GiniLoad",
    ],
    name="VEC Model",
    # agent_portrayal=agent_portrayal,
    space_drawer=False,
)

# This is required to render the visualization in the Jupyter notebook
page

Cannot show ipywidgets in text